In [ ]:
#Import the required modules

import pandas as pd
import os
import numpy as np
import re

In [ ]:
#Get the content of the csv to process the data from the config file

content_dct={}
with open('config.txt') as f:
    content=f.readlines()
    for i in content:
        var,val = i.split('=')
        content_dct[var.strip()]=val.strip()
print(content_dct)

In [ ]:
#Check whether the files loaded and dumped are of CSV extension

if(re.search('.*?\.[^csv]',content_dct['file_name']) or re.search('.*?\.[^csv]',content_dct['output_csv_name'])):
    print('Must be a CSV File')
    raise
    
if(not(re.search('.*?\.csv',content_dct['file_name']))):
    content_dct['file_name'] = content_dct['file_name']+'.csv'
    
if(not(re.search('.*?\.csv',content_dct['output_csv_name']))):
    content_dct['output_csv_name'] = content_dct['output_csv_name']+'.csv'


In [ ]:
flag=0

In [ ]:
#load the labels as dictionary
label_dct={}
for ind,lab in enumerate(content_dct['class_name'].split(',')):
    label_dct[lab]=ind
print(label_dct)

In [ ]:
#Read the Dataframe
try:
    int(content_dct['width'])
    int(content_dct['height'])
    df= pd.read_csv(content_dct['file_name'], sep=",",dtype={content_dct['xmin']:float,
                                                             content_dct['ymin']:float,content_dct['xmax']:float,
                                                             content_dct['ymax']:float})
    flag=1
except:
    df= pd.read_csv(content_dct['file_name'], sep=",",dtype={content_dct['width']:int,content_dct['height']:int,
                                              content_dct['xmin']:float,content_dct['ymin']:float,
                                              content_dct['xmax']:float,content_dct['ymax']:float})
df.head()

In [ ]:
new_df=df.copy(deep=True)
new_df.head()

In [ ]:
#Map the labels to integer
try:
    new_df=new_df.loc[new_df['class'].isin(label_dct.keys())]
    new_df['class']=new_df[content_dct['label']].map(label_dct)
    print(new_df.head())
except:
    print('The column label mentioned in the config file doesn\'t exist in the csv file')
    raise

In [ ]:
#converting the VOC points into yolo points
if(flag==0):
    try:
        new_df['x']=((df[content_dct['xmin']]+df[content_dct['xmax']])/2)/df[content_dct['width']]
        new_df['y']=((df[content_dct['ymin']]+df[content_dct['ymax']])/2)/df[content_dct['height']]
        new_df['w']=(df[content_dct['xmax']]-df[content_dct['xmin']])/df[content_dct['width']]
        new_df['h']=(df[content_dct['ymax']]-df[content_dct['ymin']])/df[content_dct['height']]
    except:
        print('The column mentioned in the config file doesn\'t exist in the csv file')
        raise
else:
    new_df['x']=((df[content_dct['xmin']]+df[content_dct['xmax']])/2)/int(content_dct['width'])
    new_df['y']=((df[content_dct['ymin']]+df[content_dct['ymax']])/2)/int(content_dct['height'])
    new_df['w']=(df[content_dct['xmax']]-df[content_dct['xmin']])/int(content_dct['width'])
    new_df['h']=(df[content_dct['ymax']]-df[content_dct['ymin']])/int(content_dct['height'])
new_df.head()

In [ ]:
final_df=pd.DataFrame({'filename':new_df['filename'],'label':new_df['class'],'x':new_df['x'],'y':new_df['y'],'w':new_df['w'],'h':new_df['h']})
final_df.head()

In [ ]:
unique_image=final_df['filename'].unique()

In [ ]:
#writting the txt file with the images of images in the specified folder
for i in unique_image:
    file_name=i.split('.')[0]+'.txt'
    row_series=final_df.loc[final_df['filename']==i,'label':'h']
    try:
        with open(content_dct['txt_file_path']+file_name,'w') as f:
            row_series.to_string(f,header=False,index=False)
    except:
        print('please check the txt file path')
        raise

In [ ]:
#writting a txt file with all the images path
try:
    with open(content_dct['txt_images_path']+'train.txt','w') as f:
        for i in unique_image:
            f.write('train/'+i+'\n')
except:
    print('please check the image names file path')
    raise

In [ ]:
#saving the csv with yolo data for future use
final_df.to_csv(content_dct['output_csv_name'],index=False)